#Задача №2 - решение задачи нелинейной регрессии с использованием полносвязной нейронной сети

Загрузим файл с данными с гугл диска

In [1]:
! gdown --id 1kD8_4C7Pwe_VUj8zNmUFmKeWGClussAr

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kD8_4C7Pwe_VUj8zNmUFmKeWGClussAr
To: /content/train_sibur_2task.parquet
100% 33.9M/33.9M [00:00<00:00, 88.9MB/s]


Импортируем необходимые библиотеки


In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
import keras
from keras.callbacks import EarlyStopping
import pandas as pd 

Откроем файл с данными и посмотрим в каком виде хранятся данные

In [3]:
data_train = pd.read_parquet("/content/train_sibur_2task.parquet", engine='pyarrow')
data_train.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1
0,32.910908,17.376350,77.557840,2.929855,gas1,20.487150,57.633085,49.245392,-44.124381,23.400064,...,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743
1,41.263782,22.419445,47.945514,-25.847472,gas2,21.461239,3.474080,49.659980,-13.553188,-13.047593,...,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716
2,25.580283,17.376350,77.654180,2.799411,gas1,20.487150,28.938295,49.245392,-44.124381,20.610679,...,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043
3,33.756900,17.376350,73.049625,2.953982,gas1,20.487150,28.932311,49.245392,-44.124381,18.107963,...,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795
4,4.223732,38.772534,48.015553,-25.843943,gas2,24.635721,12.011581,51.030938,84.244199,-17.735680,...,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082


Признак feature4 - это категориальная переменная, которая принимает 2 значения: "gas1" и "gas2". Заменим значения на числовые (0, 1) с помощью LabelEncoder() из библиотеки sklearn

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data_train['feature4'] = le.fit_transform(data_train['feature4'])
data_train.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1
0,32.910908,17.376350,77.557840,2.929855,0,20.487150,57.633085,49.245392,-44.124381,23.400064,...,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743
1,41.263782,22.419445,47.945514,-25.847472,1,21.461239,3.474080,49.659980,-13.553188,-13.047593,...,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716
2,25.580283,17.376350,77.654180,2.799411,0,20.487150,28.938295,49.245392,-44.124381,20.610679,...,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043
3,33.756900,17.376350,73.049625,2.953982,0,20.487150,28.932311,49.245392,-44.124381,18.107963,...,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795
4,4.223732,38.772534,48.015553,-25.843943,1,24.635721,12.011581,51.030938,84.244199,-17.735680,...,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082


Посмотрим, есть ли в датасете незаполненные значения (nan'ы) с помощью .isna(), если нужно, заменим их на что-то.

In [5]:
data_train.isna().sum()

feature0     0
feature1     0
feature2     0
feature3     0
feature4     0
feature5     0
feature6     0
feature7     0
feature8     0
feature9     0
feature10    0
feature11    0
feature12    0
feature13    0
feature14    0
feature15    0
feature16    0
feature17    0
feature18    0
feature19    0
feature20    0
feature21    0
feature22    0
feature23    0
feature24    0
target0      0
target1      0
dtype: int64

Пропущенных значений нет! Посмотрим есть ли дубликаты

In [6]:
data_train.duplicated().sum()

0

Импортируем дополнительные инструменты

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score

Нам необходимо предсказать параметры 'target0', 'target1'. Разделим данные на тренировочную и валидационную выборку

In [ ]:
X = data_train.drop(['target0','target1'], axis=1)
y0 = data_train['target0']
y1 = data_train['target1']

X_train, X_val, y0_train, y0_val = train_test_split(X, y0,
                                                    test_size=0.25, random_state=3984765)

In [ ]:
X_train, X_val, y1_train, y1_val = train_test_split(X, y1,
                                                    test_size=0.25, random_state=3984765)

Напишем четырех слойную полносвязную нейронную сеть в библиотеки Keras. В качестве функции активации используем ReLU, которая часто используется в регрессионных моделях. Обучим модель на наших данных, поскольку признаков 25, то input_dim=25

In [ ]:
model_0 = Sequential()
model_0.add(Dense(128, activation="relu", input_dim=25))
model_0.add(Dense(32, activation="relu"))
model_0.add(Dense(8, activation="relu"))

model_0.add(Dense(1, activation="linear"))

model_0.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01, decay=1e-3 / 200))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)


result = model_0.fit(X_train,  y0_train, validation_data=(X_val, y0_val,), epochs=15, batch_size=10, verbose=2, callbacks=[es])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/15
11507/11507 - 28s - loss: 2.8411 - val_loss: 0.3920 - 28s/epoch - 2ms/step
Epoch 2/15
11507/11507 - 24s - loss: 0.8231 - val_loss: 0.7792 - 24s/epoch - 2ms/step
Epoch 3/15
11507/11507 - 22s - loss: 0.6616 - val_loss: 0.2138 - 22s/epoch - 2ms/step
Epoch 4/15
11507/11507 - 23s - loss: 0.5302 - val_loss: 0.4533 - 23s/epoch - 2ms/step
Epoch 5/15
11507/11507 - 24s - loss: 0.4913 - val_loss: 0.3816 - 24s/epoch - 2ms/step
Epoch 6/15
11507/11507 - 21s - loss: 0.4623 - val_loss: 0.2133 - 21s/epoch - 2ms/step
Epoch 7/15
11507/11507 - 22s - loss: 0.4320 - val_loss: 0.6215 - 22s/epoch - 2ms/step
Epoch 8/15
11507/11507 - 21s - loss: 0.4066 - val_loss: 0.5628 - 21s/epoch - 2ms/step
Epoch 9/15
11507/11507 - 21s - loss: 0.3880 - val_loss: 0.2403 - 21s/epoch - 2ms/step
Epoch 10/15
11507/11507 - 22s - loss: 0.3807 - val_loss: 0.2837 - 22s/epoch - 2ms/step
Epoch 11/15
11507/11507 - 22s - loss: 0.3670 - val_loss: 0.3320 - 22s/epoch - 2ms/step
Epoch 12/15
11507/11507 - 21s - loss: 0.3523 - val_l

Посмотрим на r2_score.

In [ ]:
y0_pred = model_0.predict(X_val)
print(r2_score(y0_val, y0_pred))


1199/1199 [==============================] - 2s 1ms/step
0.999714959535414


Результат существенно лучше, чем на классической модели линейной регрессии. Посмотрим на среднюю абсолютную процентную погрешность

In [ ]:
print(mean_absolute_percentage_error(y0_val, y0_pred))

0.009253125855063313


Обучим аналогичную модель для второго признака "target1".

In [ ]:
model_1 = Sequential()
model_1.add(Dense(128, activation="relu", input_dim=25))
model_1.add(Dense(32, activation="relu"))
model_1.add(Dense(8, activation="relu"))
model_1.add(Dense(1, activation="linear"))

model_1.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001, decay=1e-3 / 200))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

result = model_1.fit(X_train,  y1_train, validation_data=(X_val, y1_val,), epochs=15, batch_size=10, verbose=2, callbacks=[es])

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


11507/11507 - 23s - loss: 2.9375 - val_loss: 8.7588 - 23s/epoch - 2ms/step
Epoch 2/15
11507/11507 - 23s - loss: 0.7172 - val_loss: 0.5037 - 23s/epoch - 2ms/step
Epoch 3/15
11507/11507 - 24s - loss: 0.5874 - val_loss: 0.4134 - 24s/epoch - 2ms/step
Epoch 4/15
11507/11507 - 24s - loss: 0.5566 - val_loss: 0.4152 - 24s/epoch - 2ms/step
Epoch 5/15
11507/11507 - 23s - loss: 0.5034 - val_loss: 0.6489 - 23s/epoch - 2ms/step
Epoch 6/15
11507/11507 - 22s - loss: 0.4908 - val_loss: 0.3962 - 22s/epoch - 2ms/step
Epoch 7/15
11507/11507 - 21s - loss: 0.4542 - val_loss: 0.3524 - 21s/epoch - 2ms/step
Epoch 8/15
11507/11507 - 21s - loss: 0.4402 - val_loss: 0.3724 - 21s/epoch - 2ms/step
Epoch 9/15
11507/11507 - 23s - loss: 0.4397 - val_loss: 0.3475 - 23s/epoch - 2ms/step
Epoch 10/15
11507/11507 - 21s - loss: 0.4197 - val_loss: 0.3333 - 21s/epoch - 2ms/step
Epoch 11/15
11507/11507 - 21s - loss: 0.4124 - val_loss: 0.4418 - 21s/epoch - 2ms/step
Epoch 12/15
11507/11507 - 21s - loss: 0.4018 - val_loss: 0.3619

Посмотрим на r2_score.

In [ ]:
y1_pred = model_1.predict(X_val)
print(r2_score(y1_val, y1_pred))

1199/1199 [==============================] - 2s 1ms/step
0.9976001243921393


В этом случае результат также оказался лучше, чем на классической модели линейной регрессии. Посмотрим на среднюю абсолютную процентную погрешность

In [ ]:
print(mean_absolute_percentage_error(y1_val, y1_pred))

0.03044211709887495


Запишем обученные модели в файлы

In [ ]:
from keras.models import load_model
model_0.save('model_0.h5')
model_1.save('model_1.h5')

Подготовим сабмит и запишем файл предикт

In [ ]:
%%writefile predict.py
import pandas as pd
from sklearn import preprocessing
from keras.models import load_model
import pathlib

le = preprocessing.LabelEncoder()



DATA_DIR = pathlib.Path(".")
MODEL_FILE_0 = pathlib.Path(__file__).parent.joinpath("model_0.h5")
MODEL_FILE_1 = pathlib.Path(__file__).parent.joinpath("model_1.h5")


def predict(df: pd.DataFrame) -> pd.DataFrame:
    """
    Вычисление предсказаний.

    Параметры:
        df:
          датафрейм, содержащий строки из тестового множества.
          Типы и имена колонок совпадают с типами и именами в ноутбуке, не содержит `np.nan` или `np.inf`.

    Результат:
        Датафрейм предсказаний.
        Должен содержать то же количество строк и в том же порядке, а также колонки `target0` и `target1`.
    """

    predictions = {}


    df['feature4'] = le.fit_transform(df['feature4'])
    FTS = df.columns

    for target, model_file in zip(["target0", "target1"], [MODEL_FILE_0, MODEL_FILE_1]):

        model = load_model(model_file)
        predictions[target] = model.predict(df[FTS])

    preds_df = pd.DataFrame(predictions, index=df.index)
    return preds_df

Writing predict.py


Запишем файл requirements.txt с версиями используемых библиотек

In [ ]:
!pip3 freeze > requirements.txt